In [22]:
import torch
import torchvision.transforms as tf
import albumentations as A
from torchvision.transforms.functional import adjust_sharpness



# 0. Normal video cloning

In [23]:
import cv2

def read_and_write_video(input_video_path, output_video_path):
    """
    Reads a video from a file and writes it to another file in MP4 format.
    
    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output video file.
    """
    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))    
    # Create the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    
    # Process each frame
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Write the frame to the output video
        out.write(frame)
        # Update the frame count
        frame_count += 1
    # Release the video capture and writer
    cap.release()
    out.release()
    # Get output video information
    out_cap = cv2.VideoCapture(output_video_path)
    out_cap.release()
    



# 1. Lower sharpness

In [24]:



def reduce_sharpness(input_video_path, output_video_path, sharpness_factor):
    """
    Reduces the sharpness of a video.
    
    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output video file.
        sharpness_factor (float, optional): Sharpness factor, ranging from 0 (blurred) to 2 (sharpened). Defaults to 0.5.
    """
    totensor = tf.ToTensor()
    # Load the video
    cap = cv2.VideoCapture(input_video_path)
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Create the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # correct format
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    
    
    # Process each frame
    frame_count = 0
    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        # Convert the frame to a PyTorch tensor
        frame_tensor = totensor(frame)
        # Adjust the sharpness of the frame 
        adjusted_frame_tensor = adjust_sharpness(frame_tensor, sharpness_factor)
        # Convert the adjusted frame back to a NumPy array
        adjusted_frame = (adjusted_frame_tensor.squeeze(0).permute(1, 2, 0) * 255).byte().numpy()
        # Write the adjusted frame to the output video
        out.write(adjusted_frame)
        frame_count += 1
        
    # Release the video capture and writer
    cap.release()
    out.release()
    
    # Print information about the output video after sharpness
    cv2.VideoCapture(output_video_path).release()
    cv2.destroyAllWindows()




In [28]:

reduce_sharpness('./videos/football.mp4','output/low_sharp.mp4', 60) # average sharpness augmentation time: 11s
# read_and_write_video('./videos/football.mp4','output/low_sharp.mp4')

# 2. Gaussian noise

In [29]:



def gaussian_noise(input_video_path, output_video_path, noise_std):
    """
    Reduces the sharpness of a video.
    
    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to the output video file.
        sharpness_factor (float, optional): Noise stdev, default 10
    """
    totensor = tf.ToTensor()
    # Load the video
    cap = cv2.VideoCapture(input_video_path)
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Create the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # correct format
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    noise_transform = A.GaussNoise(var_limit=(noise_std**2, noise_std**2), p=1.0)
    
    
    # Process each frame
    frame_count = 0
    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        # Add Gaussian noise to the frame
        noisy_frame = noise_transform(image=frame)['image']
        # Write the noisy frame to the output video
        out.write(noisy_frame)
        frame_count += 1
        
    # Release the video capture and writer
    cap.release()
    out.release()
    
    # Print information about the output video after sharpness
    cv2.VideoCapture(output_video_path).release()
    cv2.destroyAllWindows()




In [30]:
gaussian_noise('./videos/football.mp4','output/gaussian.mp4', 20)